## 🔗 Langchain ReAct agent (Ollama)


> **🚧🚧🚧🚧🚧 This is still work in progress. Not clear how Agents could be best than just LLM directly 🚧🚧🚧🚧🚧**

This notebooks explores a ReAct Agent to identify `Personal Identifyable Information` (PII)

> ℹ️ More information at: [langchain / langraph ReAct Agent](https://langchain-ai.github.io/langgraph/reference/prebuilt/#langgraph.prebuilt.chat_agent_executor.create_react_agent)


This notebooks makes use of an Ollama running server. You'll need to first run:

```bash
ollama serve &
ollama pull <some-model-you-want>
```


## ⚙️ Setup

In [4]:
# install uv
# !curl -LsSf https://astral.sh/uv/install.sh | sh

# Install Ollama
!sudo apt install lshw # for hardware detection
!curl -fsSL https://ollama.com/install.sh | sh

# Install langchain deps
!uv pip install --system \
    langgraph langchain langchain-core langchain-community langchain-ollama duckduckgo-search

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%#############################                                      50.9%
>>> Adding ollama user to render group...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Using Python 3.11.7 environment at: /usr
Audited 5 packages in 7ms


## 🦜 LLM

In [25]:
import os
import sys

# 🤯 Classic langchain... 4 different types of importing an Ollama LLM, but...
# 👉 from langchain_community.chat_models import ChatOllama
# 👉 from langchain_ollama import OllamaLLM
# 👉 from langchain_community.llms import Ollama
from langchain_ollama import ChatOllama

from langchain_core.messages import AIMessage


# ===================================== 👇 Configure as needed =================================
sys.tracebacklimit = -2
os.environ["TRACEBACK_LIMIT"] = "2"

# Or any other model capable os using tools
MODEL_ID = "llama3.2" # "jacob-ebey/phi4-tools"

OLLAMA_BASE_URL = "http://localhost:11434"
# ===================================== 👆 Configure as needed =================================


# Initialize Ollama
# llm = OllamaLLM(model=MODEL_ID, base_url=OLLAMA_BASE_URL, temperature=0.0)
json_llm = ChatOllama(model=MODEL_ID, base_url=OLLAMA_BASE_URL, temperature=0.0, format="json")
llm = ChatOllama(model=MODEL_ID, base_url=OLLAMA_BASE_URL, temperature=0.0)


prompt = "Please solve: 43x - 9 = 120"
messages = [
    ("system", "You are a helpful math assistant."),
    ("human", prompt),
]

res = llm.invoke(messages)
print(res.content)

To solve for x, we need to isolate the variable x on one side of the equation.

First, add 9 to both sides of the equation:

43x - 9 + 9 = 120 + 9
43x = 129

Next, divide both sides by 43:

43x / 43 = 129 / 43
x = 3

So, the value of x is 3.


In [18]:
import gc
import torch

del text_llm
del json_llm

gc.collect()
torch.cuda.empty_cache()

## 🤖 Agent

In [47]:
from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun
from langgraph.prebuilt import create_react_agent


# ===================================== 👇 Configure as needed =================================
TEMPLATE = """
In this chunk of markdown text:

```
{context}
```

{request}

Answer with a bullet-point list if any are found. Otherwise respond 'None'
"""
# ===================================== 👆 Configure as needed =================================


search_tool = DuckDuckGoSearchRun()

# Define a simple tool
@tool
def dummy_tool(query: str) -> str:
    """Search for information about a topic."""
    ...


tools = [search_tool]
tool_names = [t.name for t in tools]


# Create the ReAct agent
agent = create_react_agent(
    model=llm,
    tools=tools,
    # prompt=PROMPT_TEMPLATE
)

In [48]:
messages = [("human", "What is an agent in the context of LLMs?")]

# Run the agent
response = agent.invoke({"messages": messages})
print("Response:")
print(response["messages"][-1].content)

Response:
The final answer to the user's question "What is an agent in the context of LLMs?" is:

In the context of Large Language Models (LLMs), an agent refers to a software component that can perceive its environment, learn from it, reason about it, and take actions to achieve a specific goal. Agents in LLMs can be categorized into different types, including:

1. Router Agents: These agents work together with other agents to achieve a common goal.
2. Planning Agents: These agents use their planning capabilities to select which actions to take.
3. Multimodal Agents: These agents process and generate content across various formats, including audio, images, and video.

Agents in LLMs can be used to build intelligent systems that can coordinate and solve complex tasks collectively at scale. They have a wide range of applications, such as project management bots, collaborative decision-making tools, and multimodal interfaces.
